## 常用损失函数案例

In [38]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.0.0-rc0


In [39]:
cce = tf.keras.losses.CategoricalCrossentropy()
loss = cce(
    [[1.,0.,0.],[0.,1.,0.],[0.,0.,1.]],
    [[.9,.05,.05],[.05,.89,.06],[.05,.01,.94]])
print('Loss:', loss.numpy())

Loss: 0.09458993


In [40]:
# 手动计算loss是否正确

In [41]:
a = np.array([[1.,0.,0.],[0.,1.,0.],[0.,0.,1.]],)
b = np.array([[.9,.05,.05],[.05,.89,.06],[.05,.01,.94]])
np.average(-np.sum(a*np.log(b),axis=1))

0.09458991187728844

## 自定义损失函数案例

来实现MSE这个损失函数，看看是怎么自定义损失函数的

In [42]:
class MeanSquaredError(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        return tf.reduce_mean(tf.square(y_pred - y_true))
    

In [43]:
a = tf.constant([0.,0.,1.,1.])
b = tf.constant([1.,1.,1.,0.])

In [44]:
mse = MeanSquaredError()

In [45]:
mse(a,b)

<tf.Tensor: id=295, shape=(), dtype=float32, numpy=0.75>

In [46]:
# 函数式

In [47]:
def MeanSquaredError(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true))

In [48]:
mse1 = MeanSquaredError(a,b)

In [49]:
mse1

<tf.Tensor: id=299, shape=(), dtype=float32, numpy=0.75>

In [50]:
mse = tf.keras.losses.MeanSquaredError()
loss = mse([0.,0.,1.,1.],[1.,1.,1.,0.])
print('LossL',loss.numpy())

LossL 0.75


## Focal loss

In [51]:
import numpy as np
a = np.array([[1.,0.,0.],[0.,1.,0.],[0.,0.,1.]],)
b = np.array([[.9,.05,.05],[.05,.89,.06],[.05,.01,.94]])
-np.sum(a*np.log(b),axis=1)

array([0.10536052, 0.11653382, 0.0618754 ])

In [52]:
np.average(np.sum(-np.power(1-b,2)*a*np.log(b),axis=1))

0.000895471928886797

In [56]:
# 多分类的focal loss损失函数
class SparseFocalLoss(tf.keras.losses.Loss):
    
    def __init__(self,gamma=2.0, alpha=0.25, class_num=10):
        self.gamma = gamma
        self.alpha = alpha
        self.class_num = class_num
        super(SparseFocalLoss, self).__init__()
    def call(self, y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0)
        
        y_true = tf.one_hot(y_true,depth=self.class_num)
        y_true = tf.cast(y_true,tf.float32)
        
        loss = - y_true * tf.math.pow(1 - y_pred, self.gamma) * tf.math.log(y_pred)
        
        loss = tf.math.reduce_sum(loss,axis=1)
        return loss

In [57]:
focalloss = SparseFocalLoss(gamma=2.0, alpha=0.25,class_num=3)

In [58]:
focalloss(tf.constant([0,1,2]),
         tf.constant([[.9,.05,.05],[.05,.89,.06],[.05,.01,.94]]))

<tf.Tensor: id=352, shape=(), dtype=float32, numpy=0.0008954724>